In [1]:
import sys
sys.path.append("../")

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

from utils import get_bnb_config
from src.utils import set_random_seeds

In [3]:
def get_prompt(instruction: str, incontext: bool = False) -> str:
    '''Format the instruction as a prompt for LLM.'''
    if incontext:
        return f"""你是人工智慧助理，以下是用戶和人工智能助理之間的對話。你要對用戶的問題提供有用、安全、詳細和禮貌的回答。提供你兩個例子參考:
    1. USER: 翻譯成文言文：雅裏惱怒地說： 從前在福山田獵時，你誣陷獵官，現在又說這種話。ASSISTANT: 雅裏怒曰： 昔畋於福山，卿誣獵官，今復有此言。
    2. USER: 辛未，命吳堅為左丞相兼樞密使，常楙參知政事。把這句話翻譯成白話文。ASSISTANT: 初五，命令吳堅為左承相兼樞密使，常增為參知政事。
    以下的問題為文言文翻譯成白話文或白話文翻譯成文言文。請回答: USER: {instruction} ASSISTANT:"""
    else:
        return f"""你是人工智慧助理，以下是用戶和人工智能助理之間的對話。你要對用戶的問題提供有用、安全、詳細和禮貌的回答。以下的問題為文言文翻譯成白話文或白話文翻譯成文言文。USER: {instruction} ASSISTANT:"""

In [4]:
method = "lora-fine-tune"
base_model_path = "../pretrain/Taiwan-LLM-7B-v2.0-chat"
peft_path = "../checkpoint/epoch=4_ppl=3.649335366725922"

In [5]:
set_random_seeds()

# Prepare model
tokenizer = AutoTokenizer.from_pretrained(base_model_path)

bnb_config = get_bnb_config()
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)
if method == "lora-fine-tune":
    model = PeftModel.from_pretrained(model, peft_path)

device = torch.device(f"cuda:0" if torch.cuda.is_available() else "cpu")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
question = input("Please input your question: ")

prompt = get_prompt(question, incontext=True if method == "few-shot" else False)
print("Prompt:", prompt)

question_input_ids = torch.tensor(
    [tokenizer.bos_token_id] + tokenizer(prompt, add_special_tokens=False)["input_ids"]
)

model.eval()
with torch.no_grad():
    data = question_input_ids.unsqueeze(0).to(device)
    generated_tokens = model.generate(input_ids=data)
    generation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    generation = generation.replace(prompt, "").strip()
    print("Answer:", generation)

Please input your question:  翻譯成文言文：\n於是，廢帝讓瀋慶之的堂侄、直將軍瀋攸之賜瀋慶之毒藥，命瀋慶之自殺。


Prompt: 你是人工智慧助理，以下是用戶和人工智能助理之間的對話。你要對用戶的問題提供有用、安全、詳細和禮貌的回答。以下的問題為文言文翻譯成白話文或白話文翻譯成文言文。USER: 翻譯成文言文：\n於是，廢帝讓瀋慶之的堂侄、直將軍瀋攸之賜瀋慶之毒藥，命瀋慶之自殺。 ASSISTANT:
Answer: 於是，廢帝賜毒藥於慶之侄直將軍瀋攸之，令慶之自殺。
